cross attention允许输入的序列来自不同的模态

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CrossAttention(nn.Module):
    def __init__(self, embd_dim, head_nums):
        super().__init__()
        self.head_nums = head_nums
        self.head_dim = embd_dim // head_nums

        self.q = nn.Linear(embd_dim, embd_dim)
        self.k = nn.Linear(embd_dim, embd_dim)
        self.v = nn.Linear(embd_dim, embd_dim)

        self.proj = nn.Linear(embd_dim, embd_dim)
    
    def forward(self, quary, key, value, mask=None):
        batch_size, seq_len, seq_dim = quary.shape
        Q = self.q(quary).reshape(batch_size, seq_len, self.head_nums, -1).transpose(1,2)
        K = self.k(key).reshape(batch_size, seq_len, self.head_nums, -1).transpose(1,2)
        V = self.v(value).reshape(batch_size, seq_len, self.head_nums, -1).transpose(1,2)

        # 计算attention
        attn = F.softmax((Q @ K.transpose(2, 3)) / self.head_dim ** 0.5, dim=-1)
        output = attn @ V
        # 合并
        output = output.permute(0, 2, 1, 3).reshape(batch_size, seq_len, -1)
        return output


quary = torch.randn(1, 196, 768)
key = torch.randn(1, 196, 768)
value = torch.randn(1, 196, 768)
head_nums = 8
embd_dim = 768
cross_attention = CrossAttention(embd_dim, head_nums)
output = cross_attention(quary, value, key)
print(output.shape)

torch.Size([1, 196, 768])


/tmp/ipykernel_891909/1106266346.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  attn = F.softmax((Q @ K.transpose(2, 3)) / self.head_dim ** 0.5)
